<a href="https://colab.research.google.com/github/Nicolas13210/AMD_Project/blob/main/AMDproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithms for massive datasets 
## Project Report
Done by Nicolas Audoux

In [1]:
import pandas as pd
import json
import os
import hashlib
import random
from google.colab import files

Download the dataset

In [2]:
os.environ['KAGGLE_USERNAME'] = "XXXXXXXXX"
os.environ['KAGGLE_KEY'] = "XXXXXXXXX"
!kaggle datasets download -d yelp-dataset/yelp-dataset --force
!unzip yelp-dataset.zip


100% 4.06G/4.07G [00:40<00:00, 122MB/s]
100% 4.07G/4.07G [00:40<00:00, 109MB/s]
Archive:  yelp-dataset.zip
  inflating: Dataset_User_Agreement.pdf  
  inflating: yelp_academic_dataset_business.json  
  inflating: yelp_academic_dataset_checkin.json  
  inflating: yelp_academic_dataset_review.json  
  inflating: yelp_academic_dataset_tip.json  
  inflating: yelp_academic_dataset_user.json  


Pre-processing of the text:

In [3]:
def preProcess(text):
  transformedText = text.casefold()
  transformedText = transformedText.replace(',','')
  transformedText = transformedText.replace('!','')
  transformedText = transformedText.replace('?','')
  transformedText = transformedText.replace('.','')
  transformedText = transformedText.replace('\n','')
  transformedText = transformedText.replace('  ',' ')
  return transformedText

Transforming the text as a list of unique k-shingle stored as hash value of size 4

In [4]:
def getKShingles(text, k):
    global universalSet
    shingles=[]
    for i in range(0,len(text)-k):
      shingle = ''
      for j in range (k):
        shingle+=text[i+j]
      shingle = hashlib.sha1(shingle.encode('utf-8')).hexdigest()[:4]
      if shingle not in shingles:
        shingles.append(shingle)
      if shingle not in universalSet:
        universalSet.append(shingle)

    return shingles




Return a list of k-shingle for every text in the dataset and add every new shingle to the universal set

In [5]:
def createSets(dataset,k):
  sets =[]
  for i in range(len(dataset)):
    text = preProcess(dataset.get('text')[i])
    mySet= getKShingles(text,k)
    sets.append(mySet)
  return sets


Create a list which contains the n first natural number in a random order

In [6]:
def randomPermutation(n):
  permutedOrder = []
  for i in range(n):
    permutedOrder.append(random.getrandbits(128) %n)
  return permutedOrder


Compute the signature of every set of shingles in sets with respect to a random permutation

In [7]:
def computeSignature(permutation,sets):
  signature = []
  nb_sets = len(sets)
  permutation_size = len(permutation)
  for r in range(nb_sets):
    signature.append(float('inf'))
  for i in range(nb_sets):
    for j in range(permutation_size):
      if universalSet[permutation[j]] in sets[i] and permutation[j]<signature[i]:
        signature[i]=permutation[j]
  return signature

Return the  signature matrix of our sets with only one permutation of our universal set divided in m subsets (to have m number in our signatures)

In [38]:
def getSignature(sets,m):
  nb_subset = m
  subsetSize=len(universalSet)//m
  M=randomPermutation(len(universalSet))
  signatures = []
  for j in range(len(sets)):
    signatures.append([])

  for i in range(nb_subset):
    partial_signature = computeSignature(M[subsetSize*i:subsetSize*(i+1)],sets)
    for k in range(len(sets)):
      signatures[k].append(partial_signature[k])
  return signatures



Return bags of similar item if their signatures hash to the same value

In [9]:
def getSimilarPairs(signatures):
  dic={}
  ans=[]
  for sig in range(len(signatures)):
  
    hash_value = hash(signatures[sig])
    if(hash_value in dic.keys()):
      
      dic[hash_value].append(sig)
    else:
      dic[hash_value]=[sig]
  for value in dic.values():
    if len(value)>1:
      ans.append(value)
  
  return ans



Return a list of potential pair candidates

In [14]:
def getPairs(possible):
  pairs=[]
  for pair in possible:
    for i in range(len(pair)-1):
      for j in range(i+1,len(pair)):
        candidate = [pair[i],pair[j]]
        if candidate not in pairs:
          pairs.append(candidate)
  return pairs

Get the number of bands depanding on the threshold



In [10]:
def getBands(t,size):
  for b in range(size, 1,-1):
    r=size//b
    threshold = (1/b)**(1/r)
    if(threshold>t):
      return b


Divide our signature matrix in nb_band. Each band are then process throw the getSimilarPairs function to get potential candidate for high similaritie

In [11]:
def divideBand(signatures,nb_band):
  r = len(signatures[0])//nb_band
  ans=[]
  for i in range(nb_band):
    band = []
    for sig in signatures:
      band.append(tuple(sig[i*r:(i+1)*r]))
    similar=getSimilarPairs(band)
    for sim in similar:
      if sim not in ans:
        ans.append(sim)
  return getPairs(ans)

Return the similaritie between two signatures

In [12]:
def getSimilaritiesWithS(sig1,sig2):
  agree=0
  for i in range(len(sig1)):
    if sig1[i]==sig2[i] and sig1[i]!=float('inf'):
      agree+=1
  return agree/len(sig1)

Return a list of triplet with the first two elemnt of a triplet are the text and the third one, the similaritie. This similaritie must be at least equal to s.

In [13]:
def findAllSimilarities(couples,signatures,s):
  answer = []
  for couple in couples:
    sim = getSimilaritiesWithS(signatures[couple[0]],signatures[couple[1]])
    if sim>s:
      answer.append([couple[0],couple[1],sim])
  return answer

In [15]:
def getText(dataset,i):
  return dataset['text'][i]

Initialization of our variables

In [39]:
universalSet=[]
nb_documents=10000
k=5
threshold=0.6
m=100

In [40]:
dataset = pd.read_json('yelp_academic_dataset_review.json',lines=True,nrows=nb_documents)
sets=createSets(dataset,k)                    #List of all list of k-shingles representing our documents
signatures = getSignature(sets,m)        #Signature matrix for our documents (each line is a signature)
nb_band =getBands(threshold,m)
possiblePairs= divideBand(signatures,nb_band)   #List of possible pairs in our documents

In [41]:
print("There are",len((possiblePairs)),"potential similar pairs")
similarItems = findAllSimilarities(possiblePairs,signatures,threshold)
print("There are", len(similarItems), "pairs of document with a similaritie above",threshold)
print(similarItems)

There are 2410 potential similar pairs
There are 2 pairs of document with a similaritie above 0.6
[[1715, 9839, 0.85], [6523, 7971, 0.73]]


The two must similar comments

In [43]:
print(getText(dataset,1715))
print('----------------------------------------------')
print(getText(dataset,9839))

Fun place, inviting interior and a pleasure to hang outside with the couches, etc. brunch food was very good, and beer selection exceptional.  Waiter, however, was a disaster.  Was jerky to us from the beginning, (we were all sitting around asking each other why he was being such a jerk to us), did not know the menu at all, constantly disappeared on us, and was argumentative when he brought out the wrong order (twice)...and there were only 2 other tables at the time). Otherwise, would have gotten 4 stars.
----------------------------------------------
Fun place, inviting interior and a pleasure to hang outside with the couches, etc. brunch food was very good, and beer selection exceptional.  Waiter, however, was a disaster.  Was jerky to us from the beginning (we were all sitting around asking each other why he was being such an asshole to us), did not know the menu at all, constantly disappeared on us, and was argumentative when he brought out the wrong order (twice).  Otherwise, woul